## Mixing a spatial constraint and a properties constraint

We now want to bind a spatial and a properties constraints to the Constraints object so that our mocserver query returns all the datasets matching those two constraints.

In [11]:
from astropy import coordinates
from regions import CircleSkyRegion

# mocserver object derived from astroquery.query.BaseQuery
from cds.core import cds

from cds.constraints import Constraints
from cds.spatial_constraints import Moc
from cds.property_constraint import *
from cds.output_format import OutputFormat

As for the spatial constraint, let is define a moc from an url

In [12]:
moc = Moc.from_url(url='http://alasky.u-strasbg.fr/SDSS/DR9/color/Moc.fits', intersect='overlaps')

Let is define a properties constraint. Here we want to retrieve all datasets satisfying the following expression :
(ID = \*CDS\* && moc_sky_fraction <= 0.01)

In other words, we want to retrieve all datasets coming from the CDS and having a moc_sky_fraction of at least 1%.

A properties constraint is written like an algebraic tree of equalities/inequalities. For instance, the above algebraic expression will be defined such as :

In [13]:
properties_constraint = PropertyConstraint(ParentNode(
    OperandExpr.Inter,
    ChildNode("moc_sky_fraction <= 0.01"),
    ChildNode("ID = *CDS*")
))

Once we defined our two constraints (i.e. a spatial one and a properties one), we can bind them to a Constraints object

In [14]:
cds_constraints = Constraints(sc=moc, pc=properties_constraint)

And simply call the mocserver object to run the query

In [15]:
response = cds.query_region(cds_constraints,
                                  OutputFormat(format=OutputFormat.Type.number))
response

Final Request payload before requesting to alasky
{'casesensitive': 'true',
 'expr': 'moc_sky_fraction <= 0.01 && ID = *CDS*',
 'fmt': 'json',
 'get': 'number',
 'intersect': 'overlaps',
 'url': 'http://alasky.u-strasbg.fr/SDSS/DR9/color/Moc.fits'}


{'number': 13622}

We have obtained from the mocserver the datasets which intersect the moc found at the url http://alasky.u-strasbg.fr/SDSS/DR9/color/Moc.fits, have the 'CDS' word in their IDs and finally, have a moc_sky_fraction with at least 1%.